In [1]:
import DnD_Dataset_Handler as dh
import pandas as pd

data_handler = dh.DND_Dataset_handler('https://raw.githubusercontent.com/oganm/dndstats/master/docs/charTable.tsv')
source = data_handler.dataset
#display(source)

In [ ]:
data = source[['justClass', 'race']]

# Get List of single class strings, not including any multiclasses
class_list = [c.split('|') for c in data['justClass'].drop_duplicates()]
single_class_list = []
for c in class_list:
    for single_c in c:
        single_class_list.append(single_c)
single_class_list = list(dict.fromkeys(single_class_list))
single_class_list.sort()

#Get List of single race strings, removing subtypes
full_race_list = list(r for r in data['race'].drop_duplicates())
full_race_list.sort()

# Adding strings for total
full_race_list.append('Total')
single_class_list.append('Total')
total_char_amount = len(data.index)

# Result Table
#race_count = {r: 0 for r in full_race_list}
#class_count = {c: 0 for c in single_class_list}
cross_count = {c:{r:0 for r in full_race_list} for c in single_class_list}
result_data = {}

# Collecting data from each row
for index, row in data.iterrows():
    cross_count['Total'][row['race']] += 1
    for c in row['justClass'].split('|'):
        cross_count[c]['Total'] += 1
        cross_count[c][row['race']] += 1
        
# Setting up the data_dict
for c in single_class_list:
    result_data[c] = []
    for r in full_race_list:
        result_data[c].append(round((cross_count[c][r]/total_char_amount)*100, 2))
result_data['Total'][len(result_data['Total'])-1] = None
        
        
# print(race_count)
# print(class_count)
# Setting up the data

# Initiating the dataframe
df = pd.DataFrame(result_data, columns = single_class_list, index=full_race_list).sort_values('Total', ascending=False)

# Displaying the data
display(df)

In [18]:
import DnD_Dataset_Handler as dh
import pandas as pd

default_dataframe_link = 'https://raw.githubusercontent.com/oganm/dndstats/master/docs/charTable.tsv'

class ClassRaceSynergy:
    
    def __init__(self, dataframe):
        if dataframe is not None:
            self.dataset = dataframe
        else:
            data_handler = dh.DND_Dataset_handler(default_dataframe_link)
            self.dataset = data_handler.dataset
        self.__total_datarow_count = len(self.dataset.index)
        
        
    def __get_table_data(self):
        
        # Get List of single class strings, not including any multiclasses
        class_list = [c.split('|') for c in self.dataset['justClass'].drop_duplicates()]
        single_class_list = []
        for c in class_list:
            for single_c in c:
                single_class_list.append(single_c)
        self.__single_class_list = list(dict.fromkeys(single_class_list))
        self.__single_class_list.sort()

        #Get List of single race strings, removing subtypes
        self.__full_race_list = list(r for r in self.dataset['race'].drop_duplicates())
        self.__full_race_list.sort()

        # Adding strings for total
        self.__full_race_list.append('Total')
        self.__single_class_list.append('Total')

        # Counting each cross value of class and race
        cross_count = {c:{r:0 for r in self.__full_race_list} for c in self.__single_class_list}

        for index, row in self.dataset.iterrows():
            cross_count['Total'][row['race']] += 1
            for c in row['justClass'].split('|'):
                cross_count[c]['Total'] += 1
                cross_count[c][row['race']] += 1
        return cross_count
        
        
    # To get a table where each synergy is counted, messured in the counts as ints
    def get_count_table(self):    
        # Setting up the data_dict
        cross_count = self.__get_table_data()
        result_data = {}
        for c in self.__single_class_list:
            result_data[c] = []
            for r in self.__full_race_list:
                result_data[c].append(cross_count[c][r])
        result_data['Total'][len(result_data['Total'])-1] = self.__total_datarow_count
        result = pd.DataFrame(result_data, columns = self.__single_class_list, index=self.__full_race_list).sort_values('Total', ascending=True)
        return result
        
        
    # To get a table where each synergy is counted, messured in percentage of the counts as floats 
    def get_percentage_table(self): 
        # Setting up the data_dict
        cross_count = self.__get_table_data()
        result_data = {}
        for c in self.__single_class_list:
            result_data[c] = []
            for r in self.__full_race_list:
                result_data[c].append(round((cross_count[c][r]/self.__total_datarow_count)*100, 2))
        result_data['Total'][len(result_data['Total'])-1] = None
        result = pd.DataFrame(result_data, columns = self.__single_class_list, index=self.__full_race_list).sort_values('Total', ascending=False)
        return result
        
        
    # To get a table where each point shows how often the race(row) picks that class(colum) in the dataset
    def get_class_picked_by_race_percentage_table(self): 
        # Setting up the data_dict
        cross_count = self.__get_table_data()
        result_data = {}
        for c in self.__single_class_list:
            if(c == 'Total'):
                break
            else:
                result_data[c] = []
                for r in self.__full_race_list:
                    if(r == 'Total'):
                        break
                    else:
                        result_data[c].append(round((cross_count[c][r]/cross_count['Total'][r]), 2))
        
        # Remove the row and colum for total
        self.__single_class_list.remove('Total')
        self.__full_race_list.remove('Total')
        
        result = pd.DataFrame(result_data, columns = self.__single_class_list, index=self.__full_race_list)
        return result

tester = ClassRaceSynergy(None)
df = tester.get_percentage_table()
display(df)

,Artificer,Barbarian,Bard,Blood Hunter,Cleric,Druid,Fighter,Monk,Paladin,Ranger,Rogue,Sorcerer,Warlock,Wizard,Total
Human,0.00,2.04,1.72,0.00,3.92,1.72,6.43,3.29,3.29,2.19,3.45,2.51,2.19,1.72,29.15
Elf,0.00,0.31,0.31,0.00,2.51,1.88,1.88,1.41,0.31,3.29,3.76,1.25,0.78,2.19,17.71
Dwarf,0.00,2.51,0.00,0.00,1.41,0.78,1.57,0.00,1.10,0.47,0.47,0.47,0.16,0.47,9.09
Dragonborn,0.00,1.57,0.16,0.00,0.16,0.00,1.88,0.00,2.35,0.47,0.31,1.25,0.94,0.16,7.99
Gnome,0.00,0.16,0.63,0.16,0.31,0.31,0.47,0.31,0.31,0.94,1.10,0.78,0.16,1.57,6.58
Halfling,0.00,0.16,0.63,0.00,0.47,0.63,0.94,0.47,0.16,0.47,2.66,0.00,0.00,0.31,6.27
Tiefling,0.00,0.16,0.63,0.00,0.47,0.00,0.63,0.16,0.47,0.00,0.78,0.94,1.72,0.31,5.49
Aasimar,0.00,0.00,0.47,0.00,0.94,0.16,0.16,0.00,0.94,0.00,0.16,0.63,0.16,0.00,3.29
Goblin,0.00,0.16,0.31,0.00,0.16,0.31,0.31,0.16,0.16,0.16,0.78,0.16,0.00,0.47,2.98
Tabaxi,0.16,0.00,0.16,0.00,0.16,0.00,0.16,0.63,0.00,0.31,0.94,0.00,0.31,0.00,2.51
